In [86]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial
from sklearn.model_selection import train_test_split

In [87]:
read_data = pd.read_csv('Datasets/kmeans_data/data.csv')
read_labels = pd.read_csv('Datasets/kmeans_data/label.csv',names=['label'],header=None)

In [88]:
read_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Columns: 784 entries, 0 to 0.667
dtypes: int64(784)
memory usage: 59.8 MB


In [89]:
read_data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
train_data, test_data = train_test_split( read_data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( read_labels, test_size=0.08, random_state=42)

In [91]:
class Kmeans_algo:
    
    def sseCalculation(self, centr_value_dict, centr_dict,read_data):
        sse_data = 0
        for i in centr_dict:
            sse_cluster = 0
            # np.sum()
            for j in centr_dict[i]:
                d = list(read_data.iloc[int(j)])
                for a,b in zip(centr_value_dict[i],d):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def initializingCent(self,read_data,K):
        m = read_data.shape[0]
        centr_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centr_value_dict[i] = read_data.iloc[r]
        return centr_value_dict
    
    def jaccardSimilarity(self,centr, d):
        intersection = len(list(set(centr).intersection(d)))
        union = (len(set(centr)) + len(set(d))) - intersection
        return float(intersection) / union

    def trainKmeans(self,read_data,K,max_iter=20,mode=1,tol=10):
        centr_value_dict = self.initializingCent(read_data,K)
        new_centr_value_dict = {}
        count = 0
        centr_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centr_value_dict.keys()):
                centr_dict[i]=[]
            for i in range(read_data.shape[0]):
                x = read_data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centr_value_dict[j])  for j in centr_value_dict]
                    idx = np.argmin(distance_measure)
                    centr_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccardSimilarity(list(x),centr_value_dict[j]) for j in centr_value_dict]
                    idx = np.argmax(distance_measure)
                    centr_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centr_value_dict[j]))  for j in centr_value_dict]
                    idx = np.argmax(distance_measure)
                    centr_dict[idx].append(i)
                
                prev_centr=dict(centr_value_dict)
                
            
            for i in centr_dict:
                if len(centr_dict[i]):
                    dps_centr = centr_dict[i]
                    centr_value_dict[i] = np.average(read_data.iloc[dps_centr],axis=0)
            
            
            current_tol=-1
            for i in centr_value_dict:
                prev_centr_point = prev_centr[i]
                new_centr_point = centr_value_dict[i]
                change = np.sum(np.absolute(new_centr_point-prev_centr_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centr_value_dict,centr_dict

In [92]:
def predict_cluster_labels(C, S, labels):
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [93]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccardSimilarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy


In [94]:
kmeans_model = Kmeans_algo()
centroids_euclidian,clusters_euclidian = kmeans_model.trainKmeans(read_data,10, max_iter=100,mode=1)

Tolerance for the Iteration  0 :  23094.684331797234
Tolerance for the Iteration  1 :  5240.536679083683
Tolerance for the Iteration  2 :  3191.135757386712
Tolerance for the Iteration  3 :  3514.3872549019607
Tolerance for the Iteration  4 :  2311.744821788397
Tolerance for the Iteration  5 :  1622.8699259043265
Tolerance for the Iteration  6 :  1043.5406714538049
Tolerance for the Iteration  7 :  983.7870681377054
Tolerance for the Iteration  8 :  1268.5248756883032
Tolerance for the Iteration  9 :  1367.826234963518
Tolerance for the Iteration  10 :  1847.5913599842677
Tolerance for the Iteration  11 :  1675.2419726510184
Tolerance for the Iteration  12 :  1401.1749239663732
Tolerance for the Iteration  13 :  875.1779782717282
Tolerance for the Iteration  14 :  762.8421123213292
Tolerance for the Iteration  15 :  880.6614264666276
Tolerance for the Iteration  16 :  1134.8541251532647
Tolerance for the Iteration  17 :  1516.356484497449
Tolerance for the Iteration  18 :  1357.7244175

In [95]:
SSE_Euclidean = kmeans_model.sseCalculation(centroids_euclidian,clusters_euclidian,read_data)
print("SSE value for Euclidean = ",SSE_Euclidean)

SSE value for Euclidean =  25322174539.645092


In [96]:
cluster_labels_euclidian = predict_cluster_labels(centroids_euclidian,clusters_euclidian,read_labels)
cluster_labels_euclidian

array([6, 5, 7, 9, 3, 2, 0, 1, 4, 0])

In [97]:
Euclidean_accuracy = accuracy(centroids_euclidian, cluster_labels_euclidian,test_data,test_labels)
Euclidean_accuracy

0.0875

In [98]:
centroids_jaccard,clusters_jaccard = kmeans_model.trainKmeans(read_data,10, max_iter=100,mode=2)
SSE_jaccard = kmeans_model.sseCalculation(centroids_jaccard,clusters_jaccard,read_data)

Tolerance for the Iteration  0 :  30936.380952380954
Tolerance for the Iteration  1 :  4806.985515300121
Tolerance for the Iteration  2 :  2212.8959369913973
Tolerance for the Iteration  3 :  1933.299470763403
Tolerance for the Iteration  4 :  1544.3152738895935
Tolerance for the Iteration  5 :  1353.1361881565113
Tolerance for the Iteration  6 :  988.0877264819841
Tolerance for the Iteration  7 :  1988.7197791207693
Tolerance for the Iteration  8 :  1195.3951103612387
Tolerance for the Iteration  9 :  903.497547001396
Tolerance for the Iteration  10 :  1171.4783921330793
Tolerance for the Iteration  11 :  1095.1364031599664
Tolerance for the Iteration  12 :  368.80328871122407
Tolerance for the Iteration  13 :  0.0


In [99]:
print("SSE value for Jaccard  = ",SSE_jaccard)

SSE value for Jaccard  =  34361687572.938736


In [100]:
cluster_labels_jaccard = predict_cluster_labels(centroids_jaccard,clusters_jaccard,read_labels)
cluster_labels_jaccard

array([1, 2, 5, 1, 0, 5, 2, 4, 7, 3])

In [101]:
Accuracy_Jaccard = accuracy(centroids_jaccard, cluster_labels_jaccard,test_data,test_labels)
Accuracy_Jaccard

0.1075

In [102]:
centroids_cossine,clusters_cossine = kmeans_model.trainKmeans(read_data,10, max_iter=100,mode=3)
SSE_cossine = kmeans_model.sseCalculation(centroids_cossine,clusters_cossine,read_data)

Tolerance for the Iteration  0 :  28614.564676616916
Tolerance for the Iteration  1 :  7797.5
Tolerance for the Iteration  2 :  9781.409090909092
Tolerance for the Iteration  3 :  7922.2752009894875
Tolerance for the Iteration  4 :  4423.8129251700675
Tolerance for the Iteration  5 :  2069.871310507674
Tolerance for the Iteration  6 :  1867.942723543318
Tolerance for the Iteration  7 :  1527.5668212105616
Tolerance for the Iteration  8 :  1310.12339869339
Tolerance for the Iteration  9 :  849.5770596107342
Tolerance for the Iteration  10 :  783.1572056303851
Tolerance for the Iteration  11 :  671.504407881373
Tolerance for the Iteration  12 :  549.0144788975022
Tolerance for the Iteration  13 :  579.510667666917
Tolerance for the Iteration  14 :  500.9188443943625
Tolerance for the Iteration  15 :  471.5259844285235
Tolerance for the Iteration  16 :  346.12230753140136
Tolerance for the Iteration  17 :  321.8975303537776
Tolerance for the Iteration  18 :  454.9559945270224
Tolerance fo

In [103]:
print("SSE value for cossine  = ",SSE_cossine)

SSE value for cossine  =  25625843325.084


In [104]:
cluster_labels_cossine = predict_cluster_labels(centroids_cossine,clusters_cossine,read_labels)
cluster_labels_cossine

array([2, 1, 7, 6, 9, 2, 5, 0, 3, 8])

In [105]:
Accuracy_cossine = accuracy(centroids_cossine, cluster_labels_cossine,test_data,test_labels)
Accuracy_cossine

0.08125

In [107]:
print("Euclidean accuracy = ",Euclidean_accuracy)
print("Jacard accuracy = ",Accuracy_Jaccard)
print("Cosine accuracy = ",Accuracy_cossine)

Euclidean accuracy =  0.0875
Jacard accuracy =  0.1075
Cosine accuracy =  0.08125


In [108]:
print("SSE value for Euclidean = ",SSE_Euclidean)
print("SSE value for Jaccard = ",SSE_jaccard)
print("SSE value for cossine  = ",SSE_cossine)

SSE value for Euclidean =  25322174539.645092
SSE value for Jaccard =  34361687572.938736
SSE value for cossine  =  25625843325.084
